In [ ]:
from google.colab import files
import io
import pandas as pd
print("Please upload your csv file.")
uploaded = files.upload()
file_name = next(iter(uploaded))
df = pd.read_csv(io.BytesIO(uploaded[file_name]))
print(f"\nSuccessfully loaded {file_name}!")
df.head()

In [ ]:
!pip install tensorflow>=2.12.0 tflite-support>=0.4.3 transformers>=4.26.0

import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import json
import pandas as pd
import os
from google.colab import files


# --- FIX: Remove duplicate rows from the DataFrame ---
print(f"Original number of rows: {len(df)}")
df = df.drop_duplicates().reset_index(drop=True)
print(f"Number of rows after removing duplicates: {len(df)}")

# 1. Preprocess the Data
label2id = {label: i for i, label in enumerate(df['label'].unique())}
id2label = {i: label for label, i in label2id.items()}
df['label_id'] = df['label'].map(label2id)

X_train, X_val, y_train, y_val = train_test_split(
    df['text'], df['label_id'], test_size=0.2, random_state=42
)

# 2. Set up the Tokenizer and Datasets
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(X_val.tolist(), truncation=True, padding=True, max_length=128)

train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train.tolist())).shuffle(1000).batch(16)
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings), y_val.tolist())).batch(16)

# 3. Set up the DistilBERT Model
num_labels = len(label2id)
model = TFDistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    from_pt=True
)

# 4. Train the Model
num_epochs = 5
num_train_steps = len(train_dataset) * num_epochs

loss = SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
model.optimizer.learning_rate.assign(1e-4)

model.fit(train_dataset, validation_data=val_dataset, epochs=num_epochs)

# === METADATA CREATION FOR ANDROID ===
print("\n--- Creating Android metadata files ---")

# Create labels.txt file
labels_content = "\n".join([id2label[i] for i in range(len(id2label))])
with open('labels.txt', 'w') as f:
    f.write(labels_content)
print("Created labels.txt")

# Create vocab.txt from tokenizer
vocab = tokenizer.get_vocab()
sorted_vocab = sorted(vocab.items(), key=lambda x: x[1])
vocab_content = "\n".join([token for token, _ in sorted_vocab])
with open('vocab.txt', 'w') as f:
    f.write(vocab_content)
print(" Created vocab.txt")

class DistilBertWrapper(tf.keras.Model):
    def __init__(self, distilbert_model):
        super().__init__()
        self.distilbert = distilbert_model

    def call(self, ids, mask, segment_ids):
        # DistilBERT doesn't use segment_ids, so we ignore it
        outputs = self.distilbert({"input_ids": ids, "attention_mask": mask})
        logits = outputs.logits
        probabilities = tf.nn.softmax(logits, axis=-1)
        return probabilities

wrapped_model = DistilBertWrapper(model)

# CRITICAL: Define serving signature with CORRECT tensor names
@tf.function(input_signature=[
    tf.TensorSpec(shape=[1, 128], dtype=tf.int32, name="ids"),  # Changed from input_ids
    tf.TensorSpec(shape=[1, 128], dtype=tf.int32, name="mask"),  # Changed from attention_mask
    tf.TensorSpec(shape=[1, 128], dtype=tf.int32, name="segment_ids")  # Changed from token_type_ids
])
def serving_function(ids, mask, segment_ids):
    probabilities = wrapped_model(ids, mask, segment_ids)
    return {'output': probabilities}

print("Model wrapper created with tensor names: 'ids', 'mask', 'segment_ids'")

# === CONVERSION CODE ===
# 5. Convert the Model to TFLite
concrete_func = serving_function.get_concrete_function()
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]

# Convert and save without metadata first
tflite_model = converter.convert()
output_filename = 'distilbert_model_clean.tflite'

with open(output_filename, 'wb') as f:
    f.write(tflite_model)

print(f"Model converted successfully: {output_filename}")
print(f" Tensor names are now BertNLClassifier-compatible!")

# Verify tensor names
print("\nVerifying tensor names...")
interpreter = tf.lite.Interpreter(model_path=output_filename)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()

print("Input tensor names:")
for i, detail in enumerate(input_details):
    print(f"  [{i}] {detail['name']}")

expected_names = ['ids', 'mask', 'segment_ids']
actual_names = [d['name'] for d in input_details]

if all(name in actual_names for name in expected_names):
    print("\n SUCCESS! All tensor names are correct!")
else:
    print("\n WARNING: Tensor names may not be correct")

# Download files in Colab
print("\n Downloading files for Android deployment...")
from google.colab import files
files.download(output_filename)
files.download('labels.txt')
files.download('vocab.txt')

In [ ]:
# Quick test function - add this to your notebook after training
def test_model(text):
    """Quick test function for immediate testing"""
    # Tokenize the input
    encoded = tokenizer([text], truncation=True, padding=True, max_length=128, return_tensors="tf")

    # Make prediction
    prediction = model(encoded)
    logits = prediction.logits[0]

    # Get predicted class and confidence
    predicted_class_id = tf.argmax(logits).numpy()
    confidence = tf.nn.softmax(logits).numpy()

    predicted_label = id2label[predicted_class_id]
    confidence_score = confidence[predicted_class_id]

    print(f"Text: '{text}'")
    print(f"Predicted: {predicted_label}")
    print(f"Confidence: {confidence_score:.4f}")
    print("All probabilities:")
    for i, prob in enumerate(confidence):
        print(f"  {id2label[i]}: {prob:.4f}")
    print("-" * 50)

# TEST YOUR MODEL NOW!
print("=== TESTING YOUR MODEL ===\n")

# Test with various inputs
test_inputs = [
    "john.smith@gmail.com",
    "555-123-4567",
    "https://www.example.com",
    "123 Oak Street, Boston MA 02101",
    "+1-800-555-0199",
    "https://github.com/user/repo",
    "contact@company.org",
    "PO Box 789, Seattle WA"
]

for text in test_inputs:
    test_model(text)

# Interactive testing - you can modify these
print("\n=== TRY YOUR OWN INPUTS ===")
print("Change the text below and run to test:")

# CHANGE THESE TO TEST YOUR OWN INPUTS:
my_test_1 = "bhoomikasundar.cs23@rvce.edu.in"
my_test_2 = "88891-60160"
my_test_3 = "https://claude.ai/chat/deaa27fe-e7b5-4129-b015-4818da3826d5"
my_test_4 = "221 B,Basavangudi,Bengaluru"

test_model(my_test_1)
test_model(my_test_2)
test_model(my_test_3)
test_model(my_test_4)